## Variable and Memory

1. [Memory references](#memory_references)
2. [Reference counting](#reference_counting)
3. [Garbage collection](#garbage_collection)
4. [Dynamic v.s. static typing](#dynamic_static_typing)
5. [Variable re-assignment](#variable_reassignment)
6. [Object mutability & immutability](#object_mutability_immutability)
7. [Shared reference](#shared_reference)
8. [Variable equality](#variable_equality)
9. [Python optimization](#python_optimization)

### Memory references <div id='memory_references'/>

- All variables are objects in Python (both primitive and non-primitive types).
- Variables are _**references**_ or _**alias**_ of memory address.

In [1]:
# Get the memory address
a = 1
print("Memory address of a: {}".format(id(a)))

# Get the hexical memory address
print("Hexical memory address of a: {}".format(hex(id(a))))

Memory address of a: 140715366623024
Hexical memory address of a: 0x7ffad9712730


---

### Reference counting <a name='reference_counting'></a>

Reference count will increase under these cases:
- Assignment operator
- Argument passing
- Appending an object to a list (object's reference count will be increased).

In [2]:
import sys

> Immutable types: Strings, numbers and other immutable types point to the same object in memory. (not necessarily)

In [3]:
a = 1
print(sys.getrefcount(a)-1)

b = 'hi'
print(sys.getrefcount(b)-1)

2359
11


> Mutable types: Lists, dictionaries and other mutable types point to different objects in memory.

In [4]:
c = []
print(sys.getrefcount(c)-1)

d = {"key": 7}
print(sys.getrefcount(d)-1)

1
1


---

### Garbage collection <a name='garbage_collection'></a>

Garbage collection is handles by PMM (Python Memory Management) in two approaches:

- Reference counting: Destroy the object and reclaim memory once the ref_count hits 0.
- Garbage collector: Aims to handle the circular reference that reference counting does not manage to cope with.

In [5]:
import gc
import sys
import ctypes

In [6]:
# Return if an object is in garbage collector or not
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return 'Object exists'
    return 'Not found'

In [7]:
class A:
    def __init__(self):
        # Pass A's instances to B
        self.b = B(self)
        print('A: self: {}, b: {}'.format(hex(id(self)), hex(id(self.b))))

class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {}, a: {}'.format(hex(id(self)), hex(id(self.a))))

In [8]:
# Disable garbage collector
gc.disable()

obj_a = A()

B: self: 0x1fe799a3550, a: 0x1fe799a32e0
A: self: 0x1fe799a32e0, b: 0x1fe799a3550


In [9]:
a_id = id(obj_a)
print('Memory address of obj a: {}'.format(a_id))

b_id = id(obj_a.b)
print('Memory address of obj b: {}'.format(b_id))

Memory address of obj a: 2192473469664
Memory address of obj b: 2192473470288


In [10]:
print('Ref count of a:', ctypes.c_long.from_address(a_id).value)
print('Ref count of b:', ctypes.c_long.from_address(b_id).value)

Ref count of a: 2
Ref count of b: 1


In [11]:
# Trash obj_a
obj_a = None

# Circular reference
print(object_by_id(a_id))
print(object_by_id(b_id))

Object exists
Object exists


In [12]:
# Enable garbage collector
gc.collect()

# Circular reference is removed
print(object_by_id(a_id))
print(object_by_id(b_id))

Not found
Not found


---
### Dynamic v.s. static typing <a name='dynamic_static_typing'></a>

- Static typing: When declaring, a data type is associated with the variable name.
- Dynamic typing: When declaring, a variable is purely a reference to an object, no matter what type of object it is pointing to.

---
### Variable re-assignment <a name='variable_reassignment'></a>

When re-assigning a variable, it is actually just creating a new object in memory (or for some object types, find if the object is already there) and let the variable point to the new address.

In [13]:
a = 10
print(hex(id(a)))

a = 15
print(hex(id(a)))

0x7ffad9712850
0x7ffad97128f0


In [14]:
b = 10
print(hex(id(b)))  # Pointing to the same adress of which a was pointing to when a was assigned to 10

0x7ffad9712850


---

### Object mutability & immutability <a name='object_mutability_immutability'></a>

- Mutable: An object whose internal state can be changed, e.g. _**list**_, _**dict**_, _**set**_.
- Immutable: An object whose internal state cannot be changed, e.g. _**number**_, _**string**_, _**tuple**_, _**frozen set**_.

---

### Shared reference <a name='shared_reference'></a>

Two or more variables referencing/pointing to the same object in memory (i.e. haveing the same memory address). Note that for mutable objects, PMM will never create shared references since modifying one object will also modify the other one who is pointing to the shared address.

---
### Variable equality <a name='variable_equality'></a>

- Compare memory address: **is**, **not is**.
- Compare object internal data: **==**, **!=**.


---

### Python optimization <a name='python_optimization'></a>

> Number interning: CPython preloads/caches a global list of integers in the range [-5, 256] for optimization, so everytime a new variable is assigned to these numbers, it will point to a same address.

In [15]:
a = 256
b = 256
print('Are two numbers pointing to the same address?:', (a is b))

Are two numbers pointing to the same address?: True


In [16]:
a = -6
b = -6
print('Are two numbers pointing to the same address?:', (a is b))

Are two numbers pointing to the same address?: False


> String interning: Some strings will be automatically interned, mainly **identifiers** and **string literals that look like identifiers (start with \_  or a letter and only contain \_, letters and numbers)**.<br>
    This way, the memory address will be compared instead of the internal data (i.e. string itself in this case), which will improve performance dramatically.

In [17]:
# Strings look like identifiers
a = 'hello'
b = 'hello'
print('Are two strings pointing to the same address?:', (a is b))

a = 'this_is_something_trying_to_look_like_an_identifier'
b = 'this_is_something_trying_to_look_like_an_identifier'
print('Are two strings pointing to the same address?:', (a is b))

Are two strings pointing to the same address?: True
Are two strings pointing to the same address?: True


In [18]:
# Strings not like identifiers
a = 'hello world'
b = 'hello world'
print('Are two strings pointing to the same address?:', (a is b))

Are two strings pointing to the same address?: False


It is also possible to force a string to be interned using **sys.intern()**.

In [19]:
a = sys.intern('hello world')
b = sys.intern('hello world')
print('Are two strings pointing to the same address?:', (a is b))

Are two strings pointing to the same address?: True
